In [41]:


import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
# Path to the dataset root
dataset_path = '/Users/dr.ake/Documents/GitHub/Deepfake-AI-SUTD/processed_dataset_frame'

# Define the image transformations
im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms = transforms.Compose([
    transforms.Resize((im_size, im_size)),  # Resize all images to a fixed size
    transforms.ToTensor(),                  # Convert images to PyTorch tensors
    transforms.Normalize(mean, std)         # Normalize the tensor images
])

# Loading the dataset using ImageFolder
dataset = ImageFolder(dataset_path, transform=transforms)

# Splitting the dataset into train and validation subsets
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Creating data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=2)

# Example: Checking class names and some dataset info
print("Classes:", dataset.classes)
print("Number of train samples:", len(train_dataset))
print("Number of validation samples:", len(valid_dataset))


Classes: ['fake', 'real']
Number of train samples: 7634
Number of validation samples: 1909


In [42]:
# #Model with feature visualization
from torch import nn
from torchvision import models
# class Model(nn.Module):
#     def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
#         super(Model, self).__init__()

#         model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
#         self.model = nn.Sequential(*list(model.children())[:-2])
#         self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
#         self.relu = nn.LeakyReLU()
#         self.dp = nn.Dropout(0.4)
#         self.linear1 = nn.Linear(2048,num_classes)
#         self.avgpool = nn.AdaptiveAvgPool2d(1)
#     def forward(self, x):
#         batch_size,seq_length, c, h, w = x.shape
#         x = x.view(batch_size * seq_length, c, h, w)
#         fmap = self.model(x)
#         x = self.avgpool(fmap)
#         x = x.view(batch_size,seq_length,2048)
#         x_lstm,_ = self.lstm(x,None)
#         return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()

        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional=bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(hidden_dim, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        # x expected to be in shape [batch_size, channels, height, width]
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(x.size(0), -1)  # Flatten the features out
        x_lstm, _ = self.lstm(x.unsqueeze(1))  # Add a sequence dimension
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim=1)))


In [43]:
# import torch
# import numpy as np
import torchvision.models as models
from torchvision.models import ResNeXt50_32X4D_Weights

model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)



In [44]:
import torch
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=0)

print(torch.cuda.is_available())
for data, target in train_loader:
    print(data.shape, target.shape)
    break
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

False
torch.Size([4, 3, 112, 112]) torch.Size([4])
Using device: cpu


In [45]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [46]:
import torch

def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    model.to(device)  # Ensure model is on the correct device
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_predictions = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            try:
                feature_map, outputs = model(inputs)
            except Exception as e:
                print(f"Error during model forward pass: {e}")
                continue

            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_predictions / len(train_loader.dataset)

        # Validation
        valid_loss, valid_acc = validate_model(model, valid_loader, criterion, device)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, '
              f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_acc:.4f}')

def validate_model(model, valid_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            try:
                feature_map, outputs = model(inputs)
            except Exception as e:
                print(f"Error during model forward pass in validation: {e}")
                continue

            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

    total_loss = running_loss / len(valid_loader.dataset)
    total_acc = correct_predictions / len(valid_loader.dataset)
    return total_loss, total_acc

# Ensure your device setting is correct
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Initialize your model, criterion, and optimizer as before
model = Model(num_classes=len(dataset.classes))
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Call the training function
num_epochs = 10
train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs, device)


Using device: cpu
